In [197]:
#Die ganzen packages die für den versuch verwendet werden

import numpy as np #numpy ist ein package mit allen mathematischen ausdrücken/funktionen/operationen/etc. 
from uncertainties import ufloat #Uncertainties ist für das automatische berrechnen von unsicherheiten
from uncertainties.umath import * #Importieren von rechnen mit uncertainties (addition/multiplikation/etc.)
from uncertainties import unumpy as unp #kann arrays von unsicherheiten verstehen (also eine liste, statt jede zahl einzeln einzutippen)
from IPython.display import display, Math # Um ergebnisse schöner anzuzeigen. display() "ersetzt" quasi print(), und lässt latex fromatierung zu

#funktion für mittwlert mit fehler defnieren (um später ein uarray mit unsicherheiten direkt zu einem mittelwert zu machen)
def mean(values):
    nom = unp.nominal_values(values)
    std_mean = np.std(nom, ddof=1) / np.sqrt(len(nom))
    return ufloat(np.mean(nom), std_mean)


unsicherheit = 0.1 #mm?   #Ich nehme mal an das die gemessene unsicherheit immer gleich bleibt, also brauchen wir die nur einmal

Brennweite von Linsen

In [ ]:
#Konvexlinse Brennweite messen


####################################################################################
#Punkt a,

#werte liste für die gegenstands/bildweite (entfernung von linse) g/b:

g_1_werte = np.array([1, 2, 3, 4, 5])   #mm?
b_1_werte = np.array([1, 2, 3, 4, 5])   #mm?

#unsicherheitsabelle mit einzelnen werten erstellen:

g = unp.uarray(g_1_werte, unsicherheit)
b = unp.uarray(b_1_werte, unsicherheit)

#formel für brennweite f:

f = 1/(1/b+1/g)       #mm?

f_nom = unp.nominal_values(f)
f_std = unp.std_devs(f)

#-----------------------------------------------------------------------------------
#ab hier nur noch um die daten schon zu darzustellen:

#Latex tabelle definieren
table = r"""
\begin{array}{c|c}
\textbf{} & \textbf{Brennweite} ~f~[\mathrm{mm?}] \\ \hline
"""
#für jedes i-te paar (nom,std) in der numerierten liste [(1,0.1),(2,0.1),(3,0.1)...] fügen wir der vorher definierten tabelle einen neuen eintrag ein:
#(table += rf"{i-ter eintrag} & wertepaar"). Wenn keine i mehr übrig sind hört der loop mit table += r"\end{array}" auf.
for i, (nom, std) in enumerate(zip(f_nom, f_std)):
    table += rf"{i} & ({nom:.2f} \pm {std:.2f}) \\ "
table += rf"\\ {{mittel}} & ({unp.nominal_values(mean(f)):.2f} \pm {unp.std_devs(mean(f)):.2f})" #bevor der loop endet wird noch in die letzte zeile der mittelwert dazugegeben
table += r"\end{array}"

#Tabelle anzeigen lassen (printen)
display(Math(table))


display(Math(rf"{{\frac{1}{{f}}}} = D = ({1/mean(f):.2f}~\mathrm){{mm}}"))
#-----------------------------------------------------------------------------------

####################################################################################








####################################################################################
#Punkt b, Besselverfahren

#werte liste für e (gegenstand-schirm abstand) und d = d_2 - d_1 (verschiebung/symmetrische linsenposition):


e = unp.uarray([1, 2, 3, 4, 5], unsicherheit) #mm?
d_1 = unp.uarray([1, 2, 3, 4, 5], unsicherheit) #mm?
d_2 = unp.uarray([1, 2, 3, 4, 5], unsicherheit) #mm?

d = d_2-d_1

#formel für brennweite f:

f_1 = (1/4)*(e-((d**2)/e))       #mm?

f_nom_1 = unp.nominal_values(f_1)
f_std_1 = unp.std_devs(f_1)

display(Math(r"\mathrm{\textbf{Besselverfahren}}"))
table = r"""
\begin{array}{c|c}
\textbf{} & \textbf{Brennweite} ~f~[\mathrm{mm?}] \\ \hline
"""
for i, (nom_1, std_1) in enumerate(zip(f_nom_1, f_std_1)):
    table += rf"{i} & ({nom_1:.2f} \pm {std:.2f}) \\ "
table += rf"\\ {{mittel}} & ({unp.nominal_values(mean(f_1)):.2f} \pm {unp.std_devs(mean(f_1)):.2f})"
table += r"\end{array}"

display(Math(table))

display(Math(rf"{{\frac{1}{{f}}}} = D = ({1/mean(f_1):.2f}~\mathrm){{\frac{1}{{mm}}}}"))

####################################################################################



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
#Konkavlinse Brennweite messen

#Konvexlinse daten
g_1_werte = np.array([1, 2, 3, 4, 5])   #mm?
b_1_werte = np.array([1, 2, 3, 4, 5])   #mm?

#Konkavlinse daten
d_werte = np.array([10, 12, 13, 14, 15]) #Abstand der beiden linsen
b_2_werte = np.array([1, 2, 3, 4, 5]) #abstand zum schirm

d = unp.uarray(d_werte, unsicherheit)
b_2 = unp.uarray(b_2_werte, unsicherheit)

g_1 = unp.uarray(g_1_werte, unsicherheit) 
b_1 = unp.uarray(b_1_werte, unsicherheit)

g_2 = -(b_1-d)

f = 1/(1/b_2+1/g_2)       #mm?

f_nom = unp.nominal_values(f)
f_std = unp.std_devs(f)

#-----------------------------------------------------------------------------------
table = r"""
\begin{array}{c|c}
\textbf{} & \textbf{Brennweite} ~f~[\mathrm{mm?}] \\ \hline
"""
for i, (nom, std) in enumerate(zip(f_nom, f_std)):
    table += rf"{i} & ({nom:.2f} \pm {std:.2f}) \\ "
table += rf"\\ {{mittel}} & ({unp.nominal_values(mean(f)):.2f} \pm {unp.std_devs(mean(f)):.2f})"
table += r"\end{array}"

display(Math(table))

display(Math(rf"{{\frac{1}{{f}}}} = D = ({1/mean(f):.2f}~\mathrm){{mm}}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Aberration


In [187]:
#Sphärische abberration

b_n = unp.uarray([1,2,3,4,5], unsicherheit) #mm?        #Achsennahe Blende
b_f = unp.uarray([1,2,3,4,5], unsicherheit) #mm?        #Achsenferne Blende

table = r"""
\begin{array}{cc}
\\ \textbf{Ferne Bildweite} ~b_f~[\mathrm{mm?}] & \textbf{Nahe Bildweite} ~b_n~[\mathrm{mm?}]\\ \hline \\
"""
for i, (bn, bf) in enumerate(zip(b_f, b_n), start=1):
    table += rf"({unp.nominal_values(bn):.2f} \pm {unp.std_devs(bn):.2f}) & "
    table += rf"({unp.nominal_values(bf):.2f} \pm {unp.std_devs(bf):.2f}) \\ "

table += r"\\ \hline \\"
table += rf"({unp.nominal_values(mean(b_n)):.2f} \pm {unp.std_devs(mean(b_n)):.2f}) & "
table += rf"({unp.nominal_values(mean(b_f)):.2f} \pm {unp.std_devs(mean(b_f)):.2f}) \\ "
table += r"\end{array}"

display(Math(table))



<IPython.core.display.Math object>

In [188]:
#Chromatische Aberration


b_r = unp.uarray([1,2,3,4,5], unsicherheit) #mm?
b_g = unp.uarray([1,2,3,4,5], unsicherheit) #mm?
b_b = unp.uarray([1,2,3,4,5], unsicherheit) #mm?


table = r"""
\begin{array}{ccc}
\\ \textbf{Rote Bildweite}~b_r~[\mathrm{mm}] & \textbf{Grüne Bildweite}~b_g~[\mathrm{mm}] & \textbf{Blaue Bildweite}~b_b~[\mathrm{mm}] \\ \\ \hline \\
"""
for i, (br, bg, bb) in enumerate(zip(b_r, b_g, b_b), start=1):
    table += rf"({unp.nominal_values(br):.2f} \pm {unp.std_devs(br):.2f}) & "
    table += rf"({unp.nominal_values(bg):.2f} \pm {unp.std_devs(bg):.2f}) & "
    table += rf"({unp.nominal_values(bb):.2f} \pm {unp.std_devs(bb):.2f}) \\"

table += r"\\ \hline \\"
table += rf"({unp.nominal_values(mean(b_r)):.2f} \pm {unp.std_devs(mean(b_r)):.2f}) & "
table += rf"({unp.nominal_values(mean(b_g)):.2f} \pm {unp.std_devs(mean(b_g)):.2f}) & "
table += rf"({unp.nominal_values(mean(b_b)):.2f} \pm {unp.std_devs(mean(b_b)):.2f}) \\ "
table += r"\end{array}"

display(Math(table))

<IPython.core.display.Math object>